In [1]:
using GLMakie

# Generate sample data
nodes = 10
edges = [(rand(1:nodes), rand(1:nodes), rand()) for _ in 1:20]  # (source, target, weight)

# Create 3D scene
scene = Scene(resolution = (800, 600))
axis = scene[Axis]()
scatterplot!(axis, rand(nodes, 3), markersize = 0.2)  # Generate random node positions

# Add weighted edges
for (src, tgt, weight) in edges
    line!(axis, [axis.plots[end].series[1].positions[src, :][1], axis.plots[end].series[1].positions[tgt, :][1]],
                [axis.plots[end].series[1].positions[src, :][2], axis.plots[end].series[1].positions[tgt, :][2]],
                [axis.plots[end].series[1].positions[src, :][3], axis.plots[end].series[1].positions[tgt, :][3]],
                linewidth = 2 * weight, color = :blue)
end

# Show the scene
display(scene)

LoadError: MethodError: no method matching getindex(::Scene, ::Type{Axis})

[0mClosest candidates are:
[0m  getindex(::Scene, [91m::Integer[39m)
[0m[90m   @[39m [35mMakie[39m [90m~/.julia/packages/Makie/iECbF/src/[39m[90m[4mscenes.jl:381[24m[39m
[0m  getindex(::Scene, [91m::Type{OldAxis}[39m)
[0m[90m   @[39m [35mMakie[39m [90m~/.julia/packages/Makie/iECbF/src/[39m[90m[4mscenes.jl:405[24m[39m
